<a href="https://colab.research.google.com/github/patsalemether-prog/triadlink-YHWH/blob/main/copy_of_untitled19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn nest-asyncio qiskit qiskit-ibm-runtime requests -q

In [ ]:
from fastapi import FastAPI, HTTPException, Depends, Header
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import hashlib
import nest_asyncio
import uvicorn
import threading # Import threading

nest_asyncio.apply()

app = FastAPI(title="Quantum Dice API")

IBM_TOKEN = "OrEjq8B7YaBao9-fdrCbu-p4QvpWZXDwnL4TEhpzCuzK"
API_KEY = "supersecretpatkey123"

# Corrected channel parameter
service = QiskitRuntimeService(channel="ibm_quantum_platform", token=IBM_TOKEN)

def verify_api_key(x_api_key: str = Header(None)):
    if x_api_key != API_KEY:
        raise HTTPException(status_code=401, detail="Invalid key")
    return x_api_key

@app.get("/quantum-roll")
async def quantum_roll(shots: int = 2048, api_key: str = Depends(verify_api_key)):
    try:
        backend = service.backend("ibm_torino")
        print(f"Backend: {backend.name} - queue: {backend.status().pending_jobs}")

        qc = QuantumCircuit(2, 2)
        qc.h(0)
        qc.cx(0, 1)
        qc.measure([0, 1], [0, 1])

        pm = generate_preset_pass_manager(optimization_level=1, backend=backend, initial_layout=[3, 4])
        tqc = pm.run(qc)

        # Corrected Sampler initialization and run call
        sampler = Sampler(mode=backend)
        job = sampler.run([tqc], shots=shots)
        job_id = job.job_id()
        print(f"Job queued! ID: {job_id} → https://quantum.ibm.com/jobs/{job_id}")

        result = job.result()
        # Corrected counts retrieval
        counts = result[0].data.c.get_counts()

        # Initialized bits list and corrected bit extraction logic
        bits = []
        for outcome, count in counts.items():
            if outcome == "00": # Assuming '00' outcome contributes '0' bits
                bits.extend(["0"] * count)
            elif outcome == "10": # Assuming '10' outcome contributes '1' bits
                bits.extend(["1"] * count)
            # Outcomes "01" and "11" are currently ignored with this logic.

        usable = len(bits)
        if usable < 256:
            raise HTTPException(status_code=500, detail=f"Only {usable} usable bits")

        bitstring = "".join(bits[:256])
        key = hashlib.sha256(bitstring.encode()).hexdigest()

        return {
            "key": key,
            "usable_bits": usable,
            "total_shots": shots,
            "job_id": job_id,
            "proof_url": f"https://quantum.ibm.com/jobs/{job_id}"
        }
    except Exception as e:
        print("Error:", str(e))
        raise HTTPException(status_code=500, detail=str(e))

# Function to run Uvicorn in a separate thread
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Create and start the thread
thread = threading.Thread(target=run_api)
thread.daemon = True # Allows the main program to exit even if the thread is still running
thread.start()

print("Uvicorn server started in a separate thread. Access it via the displayed host:port.")

qiskit_runtime_service._discover_account:WARNING:2026-02-25 23:41:26,461: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2026-02-25 23:41:29,961: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


Uvicorn server started in a separate thread. Access it via the displayed host:port.


INFO:     Started server process [11884]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [ ]:
!fuser -k 8000/tcp

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
import hashlib # Added import for hashlib

service = QiskitRuntimeService(channel="ibm_quantum_platform", token="OrEjq8B7YaBao9-fdrCbu-p4QvpWZXDwnL4TEhpzCuzK")
job = service.job("d6fohmgj9bds73cnadhg")
print("Job status:", job.status())

if job.status() == "DONE":
    result = job.result()
    counts = result[0].data.c.get_counts() # Corrected counts retrieval

    bits = [] # Initialized bits list
    for outcome, count in counts.items():
        if outcome == "00": # Corrected bit extraction logic
            bits.extend(["0"] * count)
        elif outcome == "10": # Corrected bit extraction logic
            bits.extend(["1"] * count)
        # Outcomes "01" and "11" are currently ignored with this logic.

    usable = len(bits)
    print(f"Usable bits: {usable} / 2048")

    if usable >= 256:
        bitstring = "".join(bits[:256])
        key = hashlib.sha256(bitstring.encode()).hexdigest()
        print("Key:", key)
        print("Proof: https://quantum.ibm.com/jobs/d6fohmgj9bds73cnadhg")
else:
    print("Still running... wait 5 min and rerun this cell.")

qiskit_runtime_service._discover_account:WARNING:2026-02-25 23:53:25,542: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2026-02-25 23:53:28,710: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


Job status: DONE
Usable bits: 1069 / 2048
Key: 67f022195ee405142968ca1b53ae2513a8bab0404d70577785316fa95218e8ba
Proof: https://quantum.ibm.com/jobs/d6fohmgj9bds73cnadhg
